In [1]:
import os
import sys
import time
import re
import copy
import platform
import pandas as pd
from tqdm import tqdm
from selenium import webdriver

# set utils
util_path = os.pardir
if util_path not in sys.path:
    sys.path.append(os.pardir)
from utils.helper import load_config
from crawler import Crawler, Utills


Project_PATH = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
Save_PATH = os.path.join(Project_PATH, 'jobkorea', 'result')

# load config file
CONFIG_PATH = "config.yaml"
config = load_config(os.path.join(Project_PATH, 'jobkorea', CONFIG_PATH))
raw_data = pd.read_excel('job_korea_step1.xlsx', engine='openpyxl')
init_url = config['init_url']
selen_path_dict = config['selen_path']
chrome_opt_dict = config['chrome_option']

# chrome option
chrome_opt = webdriver.ChromeOptions()
for _ in chrome_opt_dict['default']:
    chrome_opt.add_argument(_)

if platform.system() == 'Linux':
    for _ in chrome_opt_dict['headless']:
        chrome_opt.add_argument(_)
    chrome_opt.add_argument(f'user-agent={chrome_opt_dict["user_agent"]}')

#location
location = "khrrc"


if location == 'home':
    crawler = Crawler(selen_path = selen_path_dict['home'],
                        chromeOption=chrome_opt,
                        init_url=init_url,
                        solutions = config['solution'],
                        platform_string = config['platform'])
elif location == 'laptop':
    crawler = Crawler(selen_path = selen_path_dict['laptop'],
                        chromeOption=chrome_opt,
                        init_url=init_url,
                        solutions = config['solution'],
                        platform_string = config['platform'])
elif location == 'khrrc':
    crawler = Crawler(selen_path = selen_path_dict['khrrc'],
                        chromeOption=chrome_opt,
                        init_url=init_url, 
                        solutions = config['solution'],
                        platform_string = config['platform'])
elif location == 'ubuntu':
    crawler = Crawler(selen_path= selen_path_dict['ubuntu'],
                    chromeOption = chrome_opt,
                    init_url = init_url,
                    solutions = config['solution'],
                    platform_string = config['platform'])
else:
    raise NotImplementedError

In [2]:
crawler_utils = Utills(raw_data=raw_data)
raw_data.shape

(11185, 16)

In [3]:
for idx, corp_url in tqdm(enumerate(raw_data['href'])):
    if idx < 2000:
        rs = crawler.search_corporation_info(corp_url)
        raw_data.loc[idx, '매출수'] = rs['매출수']
        raw_data.loc[idx, '사원수'] = rs['사원수']
        raw_data.loc[idx, '설립일'] = rs['설립일']
        
        if idx % 500 == 99:
            time.sleep(60) 

    else:
        pass

raw_data.to_excel(os.path.join(Save_PATH, 'rs_corp_info_2000.xlsx'))


11185it [00:02, 4028.15it/s]
